In [1]:
import sys
from pathlib import Path
local_modules_path = str(Path('.').absolute().joinpath('mf6'))

sys.path.append(local_modules_path)
import numpy as np
from tempfile import TemporaryDirectory
from mf6.voronoiplus import TriangleGrid as Triangle
from shapely import Polygon
from mf6.voronoiplus import VoronoiGridPlus as Vor
from pathlib import Path
from mf6.simplemodel import SimpleModel
from mf6.headsplus import HeadsPlus as hp
import mf6.mfsimbase as mf
import shapely as shp
import pickle
import pandas as pd

vor_path = Path.home().joinpath('mf6','mf6_model' , 'vor.vor')

In [2]:
with open(vor_path, "rb") as vor_file_load:
    vor = pickle.load(vor_file_load)
hds = hp(hds_path=Path.home().joinpath('mf6','mf6_model' , 'mf6_model.hds'), vor=vor)

In [3]:
vor.plot_choropleth().show()
hds.plot_choropleth((19,0), plot_mounding=True).show()

In [3]:
vor.plot_choropleth().show()

In [2]:
"""Example simple model"""
tri = Triangle(
    model_ws=Path.cwd(),
    angle=30
)
model_hull = tri.add_circle(radius=2000, center_coords=(50, 200), )
#facility = tri.add_circle(radius=10, center_coords=(50, 150), return_only=True)
facility = tri.add_rectangle(return_only=True)

outer_buffer = shp.Polygon(facility).buffer(0.5)
inner_buffer = shp.Polygon(facility).buffer(-0.5)
tri.add_polygon(outer_buffer)
tri.add_polygon(inner_buffer)

tri.add_regions(((10, 10), (300, 200)),
                maximum_areas=(4, 1000))
tri.build()
# get bottom elevs
strike = 30  # Strike given in degrees from north
dip = 5  # Dip given in degrees from horizontal
known_point = (50, 200, 0)  # Known point (x, y, elevation)
pixel_size = 10  # Pixel size
bottom_raster_path = Path.cwd().joinpath('bottom_raster.tif')
top_raster_path = Path.cwd().joinpath('top_raster.tif')
vor = Vor(tri)
print('getting rasters')
bottom_elevs = vor.get_raster_from_strike_dip(strike, dip, known_point, pixel_size, bottom_raster_path)
top_elevs = vor.get_raster_from_strike_dip(0, 0, (0,0,200), 10, top_raster_path)
print('got rasters')
vor = Vor(tri, rasters=[bottom_raster_path, top_raster_path])
nper = 31
facility_cells_idxs = vor.get_overlapping_grid_cells(shp.Polygon(facility)).to_list()

VoronoiGrid initialized.
Getting SciPy voronoi grid
Got SciPy voronoi grid
getting lats and lons
got lats and lons
getting rasters
getting elevations for raster
got raster from strike and dip
getting elevations for raster
got raster from strike and dip
got rasters
VoronoiGrid initialized.
Getting SciPy voronoi grid
Got SciPy voronoi grid
getting lats and lons
got lats and lons
ok
well done


In [3]:

rch_trans = [np.random.random() + 4 for per in range(nper)]
rch_dict = {}
for per in range(nper):
    cell_list = []
    for cell in range(vor.ncpl):
        if cell in facility_cells_idxs:
            cell_list.append([cell, rch_trans[per]])
        else:
            cell_list.append([cell, 0.0001])

    rch_dict[per] = cell_list
"""ADD WAY TO CALC CONDUCTANCE FROM K, L, and A"""
model = SimpleModel(
    vor,
    boundary_conductance=1000,
    nper=nper,
    rch_dict=rch_dict
)

"""save grid to file"""
vor_path = model.model_output_folder_path.joinpath('vor.vor')
with open(vor_path, "wb") as vor_file:
    pickle.dump(vor, vor_file, protocol=pickle.HIGHEST_PROTOCOL)

getting nja for disu
getting iac for disu
getting ja, cl12, and hwva for disu
getting cell areas


In [25]:
mapbox_center = {"lat": vor.grid_centroid.y, "lon": vor.grid_centroid.x}
vor.plot_choropleth().update_layout(
    margin={"r": 0, "t": 20, "l": 0, "b": 0},
    mapbox_style="carto-positron",
    mapbox_zoom=16,
    mapbox_center=mapbox_center).show()

In [20]:
vor.plot_choropleth().show()

In [18]:
vor.show_selected_cells(facility_cells_idxs).show(renderer='browser')

In [15]:
model.run_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package ims...
  writing model mf6_model...
    writing model name file...
    writing package disu...
    writing package ic...
    writing package npf...
    writing package oc...
    writing package drn...
INFORMATION: maxbound in ('gwf6', 'drn', 'dimensions') changed to 322 based on size of stress_period_data
    writing package rch...
FloPy is using the following executable to run the model: ..\..\AppData\Local\flopy\bin\mf6.exe
                                   MODFLOW 6
                U.S. GEOLOGICAL SURVEY MODULAR HYDROLOGIC MODEL
                            VERSION 6.4.2 06/28/2023

   MODFLOW 6 compiled Jul 05 2023 20:31:54 with Intel(R) Fortran Intel(R) 64
   Compiler Classic for applications running on Intel(R) 64, Version 2021.7.0
                             Build 20220726_000000

This software has been approved for release by the U.S. Geological 
Survey (USGS

In [16]:
hds.plot_choropleth(stp_per_to_plot=(19,5), plot_mounding=True).show()

In [4]:
### ### DASH ### ###
### DASH app to store selected cell ids ###

import dash
from dash import dcc
from dash import html, Input, Output
import dash_bootstrap_components as dbc

app = dash.Dash()
app.layout = html.Div(
    [
        dbc.Row(
            dbc.Col(
                [
                    dcc.Graph(
                        figure=vor.plot_choropleth(),
                        className="flex-grow-1",
                        style={"height": "95vh"},
                        id="fig",
                    )
                ],
                class_name="h-100 d-flex flex-column",
                style={"height": "95vh"},
            ),
            style={"height": "95vh"},
        ),
        dcc.Store(id="selected"),
    ],
    style={"height": "95vh"},
)


@app.callback(
    Output(component_id="selected", component_property="data"),
    Input(component_id="fig", component_property="selectedData"),
    prevent_initial_callbacks=True,
)
def on_select(selectedData):
    selected_cells = []
    for cell in selectedData["points"]:
        selected_cells.append(int(cell["location"]))
        print(selected_cells)
    return selected_cells


app.run(debug=True, use_reloader=False)  # Turn off reloader if inside Jupyter

AttributeError: 'VoronoiGridPlus' object has no attribute 'cell_list'

In [15]:
from budgetplus import BudgetPlus
import paths
bud = BudgetPlus(cbb_path=model.model_output_folder_path.joinpath('mf6_model.cbc'), vor=vor)

In [25]:
rch_bud = bud.get_rch_bud()
rch_bud = (rch_bud.reset_index(drop=True))

In [4]:
from scipy.interpolate import griddata
import pandas as pd
import plotly.graph_objs as go
heads = hp(hds_path=Path.home().joinpath('mf6', 'mf6_model', 'mf6_model.hds'), vor=vor)
"""Get Voronoi Grid Centroid x,y-coords"""
centroids_xy = list(zip(vor.centroids_x, vor.centroids_y))

""""Get heads at Vornoi Grid Vertices"""
x_surface, y_surface = vor.generate_grid_coordinates(10)
xy = list(zip(x_surface, y_surface))

"""Interpolate heads at Voronoi Vertices"""
kstpkper = (19,15)  # stress period to get data from
gridded_heads = griddata(
    points=centroids_xy, values=heads.all_heads.loc[kstpkper,:]['elev'].to_list(), xi=xy, method="cubic"
)
gridded_model_botm = griddata(
    points=centroids_xy, values=vor.gdf_topbtm["bottom"], xi=xy, method="cubic"
)
xyz_heads = list(zip(x_surface, y_surface, gridded_heads, gridded_model_botm))

"""Make DataFrame of x,y,z coordinates, sort by y,
and drop any N/A values"""
df_xyz_gridded = pd.DataFrame(xyz_heads, columns=["x", "y", "head", "botm"])
df_xyz_gridded.sort_values(by=["y", "x"], inplace=True, ascending=False)
df_xyz_gridded.reset_index(drop=True, inplace=True)

In [5]:
df_xyz_gridded['mounding'] = df_xyz_gridded['head'] - df_xyz_gridded['botm']

In [7]:
"""Reshape x,y,z arrays for Surface plot"""
col_to_plot = 'head'
unqiue_y = list(set(df_xyz_gridded["y"].to_list()))
unqiue_x = list(set(df_xyz_gridded["x"].to_list()))
xcol = df_xyz_gridded["x"].to_list().count(unqiue_x[2])
yrow = df_xyz_gridded["y"].to_list().count(unqiue_y[2])
z_reshaped = df_xyz_gridded[col_to_plot].to_numpy().reshape(xcol, yrow)
mounding_reshaped = df_xyz_gridded['mounding'].to_numpy().reshape(xcol, yrow)
x_reshaped = df_xyz_gridded["x"].to_numpy().reshape(xcol, yrow)
y_reshaped = df_xyz_gridded["y"].to_numpy().reshape(xcol, yrow)
botm_reshaped = df_xyz_gridded["botm"].to_numpy().reshape(xcol, yrow)

z_min = df_xyz_gridded[col_to_plot].min()
z_max = df_xyz_gridded[col_to_plot].max()

surface_scene = go.layout.Scene(zaxis_range=[-500, 500])

fig_surf = go.Figure(
    go.Surface(
        z=z_reshaped,
        x=x_reshaped,
        y=y_reshaped,
        surfacecolor=mounding_reshaped,
        opacity=1,
        colorscale='earth',
        scene="scene",
        #hovertemplate=
        contours={
            "z": {
                "show": True,
                "start": z_min,
                "end": z_max,
                "size": 2,
                "highlight": True,
                "width": 2,
            }
        },
    )
)
fig_surf.update_scenes(patch=surface_scene)

fig_surf.show(renderer="browser")

In [6]:
def add_rectangle(
        x_dist=100,
        y_dist=100,
        origin=(0, 0),
):
    x_min, y_min = origin[0], origin[1]
    x_max, y_max = x_min + x_dist, y_min + y_dist
    polygon_coords = ((x_min, y_min), (x_min, y_max),
                      (x_max, y_max), (x_max, y_min))
    polygon = shp.Polygon(polygon_coords)

    return polygon

In [49]:
from pathlib import Path
import plotly.graph_objs as go
import pandas as pd

itu_data_path = Path.home().joinpath('Python', 'data', 'Tehaleh SSB ITU 24hr Test All ITUs.xlsx')
itu_data = pd.read_excel(itu_data_path)
itu_data = itu_data.iloc[:,1:]

In [197]:
idxx = pd.IndexSlice
data_types = itu_data.iloc[1,:].dropna().unique()
itu_names = itu_data.iloc[0,:].dropna().unique()
multi_idx = pd.MultiIndex.from_product([itu_names,data_types])
data_columns = list(itu_data.iloc[2:,:].reset_index(drop=True).index)
df_itu = pd.DataFrame(index=multi_idx, columns=data_columns)
counter=0
data_dict = {}
for name in itu_names:
    for data_type in data_types:
        df_itu.loc[idxx[name, data_type], :] = itu_data.iloc[2:,counter]
        counter += 1
fig = go.Figure()
for itu in itu_names:
    x=df_itu.loc[(itu, 'Elapsed Time (min)'), :]
    y=df_itu.loc[(itu, 'WL elevation (ft amsl)'), :]
    y0 = y.dropna().min()
    sd = [y1-y0 for y1 in y]
    data_dict[itu] = (list(x), list(y), list(sd), y0)
    fig.add_scattergl(
        x=x,
        y=y,
        name=itu
    )

In [251]:
fig.update_layout(dragmode='pan')
fig.show(renderer='browser', config={'scrollZoom': True})

In [249]:
import scipy
fig_ex = go.Figure()
for itu in list(data_dict.keys()):
    xd, yd, sd, y0 = zip(data_dict[itu])
    max_time = pd.Series(xd[0]).max()
    times_to_extend = [time for time in range(1441, int(max_time), 1)]
    times_since_off = np.array([time-1440 for time in times_to_extend])
    interpd = scipy.interpolate.interp1d(xd[0],yd[0])
    sd_recovery = interpd(times_to_extend) - y0[0]
    sd_time_since_off = interpd(times_since_off) - y0[0]
    sd_extrapolated = sd_recovery+sd_time_since_off
    y_extrapolated = y0[0] + sd_extrapolated
    fig_ex.add_scattergl(x=xd[0], y=yd[0], name=itu)
    fig_ex.add_scattergl(
        x=times_to_extend,
        y=y_extrapolated.tolist(),
        name=f'{itu} extrapolated'
    )

In [250]:
fig_ex.show(renderer='browser')

In [205]:
interpd = scipy.interpolate.interp1d(xd,yd)

In [2]:
import pickle
from pathlib import Path

fig_path = Path.home().joinpath('.azure', 'Ten-Trails-Water-Levels', 'Ten%20Trails%20Water%20Levels%20Visualization', 'sample_data', 'Ten_Trails.fig')
with open(fig_path, mode='rb') as f:
    fig = pickle.load(f)

In [8]:
fig.data[183]

Scattergl({
    'line': {'color': 'rgb(140, 86, 75)', 'width': 1.5},
    'marker': {'color': 'rgb(140, 86, 75)', 'size': 5},
    'mode': 'markers',
    'name': 'RW-1',
    'x': [1993-11-02 00:00:00, 1993-12-10 00:00:00, 1994-02-02 00:00:00, ...,
          2023-09-18 00:00:00, 2023-10-12 00:00:00, 2023-10-17 00:00:00],
    'xaxis': 'x',
    'y': [nan, nan, nan, ..., nan, nan, nan],
    'yaxis': 'y'
})

In [7]:
import sys
from pathlib import Path
import os
current_path = os.environ.get('Path', '')
qgis_path = 'C:\\OSGeo4W\\apps\\qgis\\python\\qgis'
new_path = qgis_path + os.pathsep + current_path
os.environ['PATH'] = new_path

In [8]:
import qgis

ModuleNotFoundError: No module named 'qgis'